In [1]:
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import math
import torch
from torch import nn
from sklearn import metrics

In [2]:
def imputeDataMean(scliedData, MaskMat, meanvalue):
    imputedata = []
    for i in range(len(scliedData)):
        a = np.array(scliedData[i])
        b = np.array(scliedData[i]) * np.array(MaskMat[i])
        e = b.sum(axis=0)
        c = np.array(MaskMat[0])
        d = c.sum(axis=0)
        d[d==0]=1
        mean = b.sum(axis=0)/d
        for j in range(a.shape[1]):
            if np.all(a[:,j] == -1):
                a[:,j]= meanvalue[j]
            else:
                k = a[:,j]
                k[k==-1]= e[j]
        imputedata.append(a)
    return imputedata

In [3]:
def imputeDataNearest(sliceData, meanvalue):
    imputedata = []
    for i in range(len(sliceData)):
        a = np.array(sliceData[i])
        b = pd.DataFrame(np.array(a))
        b = b.replace(-1,np.nan)
        c = b.count()
        for k in range(a.shape[1]):
            if c[k] == 0:
                b[k] = meanvalue[k]
            if c[k] == 1:
                b[k] = b[k].interpolate(method ='ffill',axis=0)
                b[k] = b[k].interpolate(method ='bfill',axis=0)
        b = b.interpolate(method ='nearest',axis=0,limit_direction ='both')
        b = b.interpolate(method ='ffill')
        b = b.interpolate(method ='bfill')
        b = b.values
        imputedata.append(b)
    return imputedata

In [4]:
def imputeLast(sliceData, meanvalue):
    imputedata = []
    for i in range(len(sliceData)):
        a = np.array(sliceData[i])
        for j in range(a.shape[1]):
            if np.all(a[:,j] == -1):
                a[:,j]= meanvalue[j]
        b = pd.DataFrame(np.array(a))
        b = b.replace(-1,np.nan)
        b = b.interpolate(method ='ffill',axis=0)
        b = b.interpolate(method ='bfill',axis=0)
        b = b.values
        imputedata.append(b)
    return imputedata

In [21]:
f = open('E:/WashU/Research/ICU/Data/train/X_train_sliced_norm.pkl','rb')
X_train_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/y_train.pkl','rb')
y_train = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_delta_mat.pkl','rb')
train_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_mask_mat.pkl','rb')
train_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/mean_norm.pkl','rb')
mean = pickle.load(f)
f.close()

In [26]:
X_train_sliced = np.array(imputeDataMean(X_train_sliced, train_delta_mat, mean))

X_train = np.array(X_train_sliced[:3600])
X_val = np.array(X_train_sliced[3600:])
train_delta = np.array(train_delta_mat[:3600])
val_delta = np.array(train_delta_mat[3600:])
Y_train = np.array(y_train[:3600])
Y_val = np.array(y_train[3600:])
X_resampled = []
y_resampled = []
delta_resampled = []
y_one_index = np.argwhere(Y_train==1).reshape(-1)
for i in np.argwhere(Y_train==0).reshape(-1):
    X_resampled.append(X_train[i])
    y_resampled.append(Y_train[i])
    delta_resampled.append(train_delta[i])
    sample_index = np.random.randint(0, len(y_one_index)-1)
    X_resampled.append(X_train[y_one_index[sample_index]])
    y_resampled.append(Y_train[y_one_index[sample_index]])
    delta_resampled.append(train_delta[y_one_index[sample_index]])
X_resampled = np.array(X_resampled)
delta_resampled = np.array(delta_resampled)

In [7]:
def grad_clipping(net, theta):
    params = [p for p in net.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [8]:
class GRUICell(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        super(GRUICell, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        def normal(shape):
            return torch.randn(size=shape)*0.0001
        def three():
            return (nn.Parameter(normal((num_inputs, num_hiddens))), 
                    nn.Parameter(normal((num_hiddens, num_hiddens))), 
                    nn.Parameter(torch.zeros(num_hiddens)))
        self.W_xz, self.W_hz, self.b_z = three() # Parameters of update gate
        self.W_xr, self.W_hr, self.b_r = three() # Parameters of reset gate
        self.W_xh, self.W_hh, self.b_h = three() # Parameters of candidate hidden state
        # Parameters of decay vector
        self.W_beta = nn.Parameter(normal((num_inputs, num_hiddens)))
        self.b_beta = nn.Parameter(torch.zeros(num_hiddens))
        # Parameters of output layer
        self.W_hq = nn.Parameter(normal((num_hiddens, num_outputs)))
        self.b_q = nn.Parameter(torch.zeros(num_outputs))

    def forward(self, X, Delta, H):
        beta = torch.exp(torch.minimum(torch.zeros(self.num_hiddens), Delta @ self.W_beta + self.b_beta))
        H = beta * H
        Z = torch.sigmoid((X @ self.W_xz) + (H @ self.W_hz) + self.b_z)
        R = torch.sigmoid((X @ self.W_xr) + (H @ self.W_hr) + self.b_r)
        H_tilde = torch.tanh((X @ self.W_xh) + ((R * H) @ self.W_hh) + self.b_h)
        H = Z * H + (1 - Z) * H_tilde
        H.detach()
        Y = torch.sigmoid(H @ self.W_hq + self.b_q)
        return Y, H

In [9]:
class GRUIModel(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, imputeMethod, scaleMethod, resampleMethod):
        super(GRUIModel, self).__init__()
        self.name = 'GRUI'
        self.num_hiddens = num_hiddens
        self.gruicell = GRUICell(num_inputs, num_hiddens, num_outputs)
        self.imputeMethod = imputeMethod
        self.scaleMethod = scaleMethod
        self.resampleMethod = resampleMethod

    def forward(self, X, Delta, H):
        if H is None:
            H = torch.zeros(X.shape[1], self.num_hiddens)
        for index in range(X.shape[0]):
            Y, H = self.gruicell(X[index], Delta[index], H)
        return Y, H

In [10]:
def Score1(method_Pred, ytest):
    score1 = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for i in range (len(ytest)):
        if (ytest[i] == 1) & (method_Pred[i] == 1):
            TP = TP + 1
        if (ytest[i] == 1) & (method_Pred[i] == 0):
            FN = FN + 1
        if (ytest[i] == 0) & (method_Pred[i] == 1):
            FP = FP + 1
        if (ytest[i] == 0) & (method_Pred[i] == 0):
            TN = TN + 1
    if ((TP == 0) & (FN == 0)):
        Se = 0
    else:
        Se = TP/(TP+FN)
        
    if ((TP == 0) & (FP == 0)):
        P = 0
    else:
        P = TP/(TP+FP)
    
    if Se > P:
        score1 = P
    else:
        score1 = Se
    return score1

In [11]:
def train_grui_model(model, X_train, y_train, train_delta_mat, X_val, y_val, val_delta_mat, batch_size, lr, num_epoch=25):
    train_loss_all, train_acc_all = [], []
    optimizer = torch.optim.SGD(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()
    threshold = torch.tensor([0.5])
    
    train_num = X_train.shape[0]
    val_num = X_val.shape[0]
    X_train = torch.from_numpy(X_train.transpose(1, 0, 2)).float()
    X_val = torch.from_numpy(X_val.transpose(1, 0, 2)).float()
    train_delta_mat = torch.from_numpy(train_delta_mat.transpose(1, 0, 2)).float()
    val_delta_mat = torch.from_numpy(val_delta_mat.transpose(1, 0, 2)).float()
    y_train = torch.from_numpy(np.array(y_train)).type(torch.LongTensor)
    y_val = torch.from_numpy(np.array(y_val)).type(torch.LongTensor)
    
    # Compute the AUC of validation set
    output, _ = model(X_val, val_delta_mat, None)
    print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    for epoch in range(num_epoch):
        print("-" * 40)
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        
        # training stage
        train_loss, train_corrects = 0, 0
        test_loss, test_corrects = 0, 0
        
        for step in range(train_num//batch_size+1):
            if (step+1)*batch_size <= train_num:
                X = X_train[:, int(step*batch_size):int((step+1)*batch_size), :]
                y = y_train[int(step*batch_size):int((step+1)*batch_size)]
                Delta = train_delta_mat[:, int(step*batch_size):int((step+1)*batch_size), :]
            else:
                X = X_train[:, int(step*batch_size):, :]
                y = y_train[int(step*batch_size):]
                Delta = train_delta_mat[:, int(step*batch_size):, :]
            
            output, _ = model(X, Delta, None)
            y_hat = torch.cat((1-output,output),dim=1)
            loss = criterion(y_hat, y)
            y_predict = (output > threshold).float() * 1
            
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model, 1)
            optimizer.step()
            train_loss += loss.item() * len(y)
            train_corrects += torch.sum(y_predict.squeeze() == y)
        
        # Compute the mean of loss and accuracy for every epoch
        train_loss_all.append(train_loss / train_num)
        train_acc_all.append(train_corrects.double().item() / train_num)
        
        print("Train Loss: {:.4f} Train Acc{:.4f}".format(train_loss_all[-1], train_acc_all[-1]))
        
        if (epoch+1) % 5 == 0:
            # Compute the AUC of validation set
            output, _ = model(X_val, val_delta_mat, None)
            y_predict = (output > threshold).float() * 1
            print('Val Auc: {:.4f}, Val Score 1: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy()), Score1(y_predict, y_val)))
    
    # save
    torch.save(model.state_dict(), 'E:/WashU/Research/ICU/modelParams/{}_{}_{}_{}_{}_{}_{}_{}.pth'.format(model.imputeMethod, 
                                                                                                          model.scaleMethod, 
                                                                                                          model.resampleMethod, 
                                                                                                          model.name, 
                                                                                                          model.num_hiddens, 
                                                                                                          batch_size, 
                                                                                                          lr, 
                                                                                                          num_epoch))
    
    return model

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUIModel(num_inputs=X_train.shape[2], num_hiddens=64, num_outputs=1, imputeMethod='Last', scaleMethod='Norm', resampleMethod='RandomBalance').to(device)
model = train_grui_model(model, X_resampled, y_resampled, delta_resampled, X_val, Y_val, val_delta, batch_size=32, lr=0.1, num_epoch=100)

Val Auc: 0.4957
----------------------------------------
Epoch 1/100
Train Loss: 0.6931 Train Acc0.6055
----------------------------------------
Epoch 2/100
Train Loss: 0.6931 Train Acc0.6609
----------------------------------------
Epoch 3/100
Train Loss: 0.6931 Train Acc0.6581
----------------------------------------
Epoch 4/100
Train Loss: 0.6931 Train Acc0.6302
----------------------------------------
Epoch 5/100
Train Loss: 0.6931 Train Acc0.5912
Val Auc: 0.7101, Val Score 1: 0.1379
----------------------------------------
Epoch 6/100
Train Loss: 0.6928 Train Acc0.5979
----------------------------------------
Epoch 7/100
Train Loss: 0.6914 Train Acc0.6373
----------------------------------------
Epoch 8/100
Train Loss: 0.6832 Train Acc0.6848
----------------------------------------
Epoch 9/100
Train Loss: 0.6477 Train Acc0.6964
----------------------------------------
Epoch 10/100
Train Loss: 0.5955 Train Acc0.7151
Val Auc: 0.7654, Val Score 1: 0.2360
-----------------------------

Train Loss: 0.5178 Train Acc0.7880
----------------------------------------
Epoch 87/100
Train Loss: 0.5179 Train Acc0.7897
----------------------------------------
Epoch 88/100
Train Loss: 0.5200 Train Acc0.7860
----------------------------------------
Epoch 89/100
Train Loss: 0.5166 Train Acc0.7909
----------------------------------------
Epoch 90/100
Train Loss: 0.5174 Train Acc0.7899
Val Auc: 0.8305, Val Score 1: 0.3617
----------------------------------------
Epoch 91/100
Train Loss: 0.5178 Train Acc0.7894
----------------------------------------
Epoch 92/100
Train Loss: 0.5168 Train Acc0.7894
----------------------------------------
Epoch 93/100
Train Loss: 0.5181 Train Acc0.7889
----------------------------------------
Epoch 94/100
Train Loss: 0.5154 Train Acc0.7925
----------------------------------------
Epoch 95/100
Train Loss: 0.5160 Train Acc0.7909
Val Auc: 0.8247, Val Score 1: 0.2624
----------------------------------------
Epoch 96/100
Train Loss: 0.5140 Train Acc0.7947
-